In [0]:
#MONTAGGIO DRIVE
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!pip install GPUtil
!pip install segmentation_models

In [0]:
import GPUtil as GPU
GPUs = GPU.getGPUs()
gpu = GPUs[0]
print(gpu.name)

Tesla K80


In [0]:
!unzip "drive/My Drive/Colab Notebooks/PROGETTO/TrayDataset.zip" -d "image-dataset/"

In [0]:
from os import listdir

from skimage.io import imread_collection, imshow, imread
from sklearn.model_selection import train_test_split

import numpy as np
from numpy.random import uniform

import keras.backend as K
import segmentation_models as sm
from segmentation_models import Unet
from segmentation_models.losses import bce_jaccard_loss, dice_loss
from segmentation_models.metrics import iou_score
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [0]:

#x_train = imread_collection('image-dataset/TrayDataset/XTrain/*.jpg')
#x_test = imread_collection('image-dataset/TrayDataset/XTest/*.jpg')

#y_train = imread_collection('image-dataset/TrayDataset/yTrain/*.png')
#y_test = imread_collection('image-dataset/TrayDataset/yTest/*.png')

#imshow(x_test[1])

# load your data
x_train = []
for image in listdir('image-dataset/TrayDataset/XTrain/'):
    x_train.append((image, imread('image-dataset/TrayDataset/XTrain/'+image), image[0]))
                                       
y_train = []
for image in listdir('image-dataset/TrayDataset/yTrain/'):
    y_train.append((image, imread('image-dataset/TrayDataset/yTrain/'+image), image[0]))

In [0]:
print(x_train[0])

In [0]:

#x_train = np.array(x_train)
#y_train = np.array(y_train)
# ordino i due vettori e ricontrollo
x_train.sort(key = lambda x: x[0])
y_train.sort(key = lambda x: x[0])
x_Train = [i[1] for i in x_train]
y_Train = [i[1] for i in y_train]

# mi prendo i label per fare una stratificazione
x_label = [i[2] for i in x_train]

print(x_label)

del x_train
del y_train
# riduco in 0-1

x_Train = np.array(x_Train).astype('float32')/255
# splitto in training e validation -> aggiunta stratificazione sul label elemento[0] del nome img
X_train, X_val, y_train, y_val = train_test_split(x_Train, y_Train, test_size=0.25, random_state=17, stratify = x_label)
X_train = np.array(X_train)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_val = np.array(y_val)

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',

In [0]:
def one_hot_it(label, label_values):
    """
    Convert a segmentation image label array to one-hot format
    by replacing each pixel value with a vector of length num_classes
    # Arguments
        label: The 2D array segmentation image label
        label_values
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of num_classes
    """
    #print('Start One-Hot Vectorizing...')
    semantic_map = []
    for colour in label_values:
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)
    #print('Done..')
    return semantic_map

background=[0,0,0]
tray=[1,1,1]
cutler=[2,2,2]
form=[3,3,3]
straw=[4,4,4]
meatball=[5,5,5]
beef=[6,6,6]
roastlamb=[7,7,7]
beeftomatocasserole=[8,8,8]
ham=[9,9,9]
bean=[10,10,10]
cucumber=[11,11,11]
leaf=[12,12,12]
tomato=[13,13,13]
boiledrice=[14,14,14]
beefmexicanmeatballs=[15,15,15]
spinachandpumpkinrisotto=[16,16,16]
bakedfish=[17,17,17]
gravy=[18,18,18]
zucchini=[19,19,19]
carrot=[20,20,20]
broccoli=[21,21,21]
pumpkin=[22,22,22]
celery=[23,23,23]
sandwich=[24,24,24]
sidesalad=[25,25,25]
tartaresauce=[26,26,26]
jacketpotato=[27,27,27]
creamedpotato=[28,28,28]
bread=[29,29,29]
margarine=[30,30,30]
soup=[31,31,31]
apple=[32,32,32]
cannedfruit=[33,33,33]
milk=[34,34,34]
vanillayogurt=[35,35,35]
jelly=[36,36,36]
custard=[37,37,37]
lemonsponge=[38,38,38]
juice=[39,39,39]
applejuice=[40,40,40]
orangejuice=[41,41,41]
water=[42,42,42]

#We dont want to classify unlabelled. it will only make our dataset more unbalanced than it already is
label_values = [background,tray,cutler,form,straw,meatball,beef,roastlamb,beeftomatocasserole,ham,bean,cucumber,leaf,tomato,boiledrice,
beefmexicanmeatballs,spinachandpumpkinrisotto,bakedfish,gravy,zucchini,carrot,broccoli,pumpkin,celery,sandwich,sidesalad,
tartaresauce,jacketpotato,creamedpotato,bread,margarine,soup,apple,cannedfruit,milk,vanillayogurt,jelly,custard,lemonsponge,
juice,applejuice,orangejuice,water]

y_train_norm = [one_hot_it(y_train[i], label_values) for i in range(len(y_train))]
labels_train = np.array(y_train_norm)

y_val_norm = [one_hot_it(y_val[i], label_values) for i in range(len(y_val))]
labels_val = np.array(y_val_norm)

#y_test_norm = [one_hot_it(y_test[i],label_values) for i in range(len(y_test))]
#labels_test = np.array(y_test)

In [0]:
del y_train_norm
del y_val_norm
del y_train
del y_val

In [0]:

def iou_coef(y_true, y_pred):
  smooth=1
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3]) #moltiplicazione element 
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou

# IoU = (|X &amp; Y|)/ (|X or Y|)
def iou_coef_loss(y_true, y_pred):
  return 1-iou_coef(y_true, y_pred)

Using TensorFlow backend.


In [0]:

for i in range (0, 50):
  lr = 10**uniform(-2.6, -3.1)

  model = Unet('vgg16', 
            input_shape=(256, 416, 3), 
            encoder_freeze=True, 
            encoder_weights='imagenet',
            classes=43, 
            activation='softmax') #softmax = multiclass segmentation

  adam = Adam(learning_rate = lr)

  model.compile(loss=bce_jaccard_loss, optimizer=adam, metrics=[iou_coef])

  print("LR=", lr)
  results = model.fit(x = X_train,
                    y = labels_train,
                    epochs=20,
                    batch_size=16,
                    validation_data=[X_val, labels_val],
                    verbose=1)

LR= 0.001788746441245911
Train on 930 samples, validate on 311 samples
Epoch 1/20
930/930 [==============================] - 40s 43ms/step - loss: 0.9789 - iou_coef: 0.5254 - val_loss: 1.6218 - val_iou_coef: 0.0218
Epoch 2/20
930/930 [==============================] - 38s 41ms/step - loss: 0.8823 - iou_coef: 0.8159 - val_loss: 1.1308 - val_iou_coef: 0.2705
Epoch 3/20
930/930 [==============================] - 38s 41ms/step - loss: 0.7655 - iou_coef: 0.8698 - val_loss: 0.9147 - val_iou_coef: 0.6519
Epoch 4/20
930/930 [==============================] - 38s 41ms/step - loss: 0.5962 - iou_coef: 0.9229 - val_loss: 0.8207 - val_iou_coef: 0.4393
Epoch 5/20
930/930 [==============================] - 38s 41ms/step - loss: 0.4878 - iou_coef: 0.9424 - val_loss: 0.7013 - val_iou_coef: 0.8520
Epoch 6/20
930/930 [==============================] - 38s 41ms/step - loss: 0.4354 - iou_coef: 0.9576 - val_loss: 0.5666 - val_iou_coef: 0.9188
Epoch 7/20
930/930 [==============================] - 38s 41ms/st

KeyboardInterrupt: ignored

In [0]:
#TRAINING
lr = 0.0011836219838358216

BACKBONE = 'vgg16'
preprocess_input = sm.get_preprocessing(BACKBONE)

model = Unet(BACKBONE, 
            input_shape=(256, 416, 3), 
            encoder_freeze=True, 
            encoder_weights='imagenet',
            classes=43, 
            activation='softmax') #softmax = multiclass segmentation

adam = Adam(learning_rate = lr)

model.compile(loss=bce_jaccard_loss, optimizer=adam, metrics=[iou_coef])

#checkpoint per salvare il modello quando l'accuracy sul validation è max
filepath = "/content/drive/My Drive/Colab Notebooks/PROGETTO/Unet(vgg16)OPTIMIZED.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_iou_coef', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

results = model.fit(x = X_train,
                    y = labels_train,
                    epochs=50,
                    batch_size=16,
                    validation_data=[X_val, labels_val],
                    callbacks=callbacks_list,
                    verbose=1)


Train on 930 samples, validate on 311 samples
Epoch 1/50
930/930 [==============================] - 40s 43ms/step - loss: 1.0105 - iou_coef: 0.3340 - val_loss: 1.1923 - val_iou_coef: 0.2072

Epoch 00001: val_iou_coef improved from -inf to 0.20717, saving model to /content/drive/My Drive/Colab Notebooks/PROGETTO/Unet(vgg16)OPTIMIZED.hdf5


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


Epoch 2/50
930/930 [==============================] - 38s 41ms/step - loss: 0.8938 - iou_coef: 0.7853 - val_loss: 0.9750 - val_iou_coef: 0.5504

Epoch 00002: val_iou_coef improved from 0.20717 to 0.55042, saving model to /content/drive/My Drive/Colab Notebooks/PROGETTO/Unet(vgg16)OPTIMIZED.hdf5
Epoch 3/50
930/930 [==============================] - 38s 41ms/step - loss: 0.8316 - iou_coef: 0.8450 - val_loss: 0.9567 - val_iou_coef: 0.4076

Epoch 00003: val_iou_coef did not improve from 0.55042
Epoch 4/50
930/930 [==============================] - 38s 41ms/step - loss: 0.7550 - iou_coef: 0.8789 - val_loss: 0.8743 - val_iou_coef: 0.5651

Epoch 00004: val_iou_coef improved from 0.55042 to 0.56514, saving model to /content/drive/My Drive/Colab Notebooks/PROGETTO/Unet(vgg16)OPTIMIZED.hdf5
Epoch 5/50
930/930 [==============================] - 38s 41ms/step - loss: 0.6162 - iou_coef: 0.9182 - val_loss: 0.7364 - val_iou_coef: 0.7717

Epoch 00005: val_iou_coef improved from 0.56514 to 0.77167, sav

In [0]:
#MODEL SAVE
BACKBONE = 'vgg16'
preprocess_input = sm.get_preprocessing(BACKBONE)

model = Unet(BACKBONE, 
            input_shape=(256, 416, 3), 
            encoder_freeze=True, 
            encoder_weights='imagenet',
            classes=43, 
            activation='softmax') #softmax = multiclass segmentation

model.load_weights('/content/drive/My Drive/Colab Notebooks/PROGETTO/Unet(vgg16)OPTIMIZED.hdf5')

model.save('/content/drive/My Drive/Colab Notebooks/PROGETTO/Unet(vgg16)OPTIMIZED.h5')

In [0]:
#DEALLOCA
del labels_val
del labels_train
del X_train
del X_val

In [0]:
#VALUTAZIONE SU TEST SET
x_test = imread_collection('image-dataset/TrayDataset/XTest/*.jpg')
y_test = imread_collection('image-dataset/TrayDataset/yTest/*.png')

test_set = []
for img in x_test:
  test_set.append(img)

test_set = np.array(test_set).astype('float32')/255

y_test_norm = [one_hot_it(y_test[i],label_values) for i in range(len(y_test))]
labels_test = np.array(y_test_norm)

del y_test_norm

In [0]:
#Carico i pesi per la prediction
model = Unet('vgg16', 
            input_shape = (256, 416, 3),
            classes = 43, 
            encoder_freeze = True,
            encoder_weights = 'imagenet',
            activation = 'softmax') #softmax = multiclass segmentation

model.compile('Adam', 
              loss = bce_jaccard_loss, 
              #loss="categorical_crossentropy",
              metrics = [iou_coef])

model.load_weights("/content/drive/My Drive/Colab Notebooks/PROGETTO/Unet(vgg16-jaccard)MyIOU-Stratified.hdf5")


In [0]:
#Prediction
scores = model.evaluate(test_set, labels_test, verbose=1)
print(scores[1])

In [0]:
#Visualizzo predizioni
def colour_code_segmentation(image, label_values):
    """
    Given a 1-channel array of class keys, colour code the segmentation results.
    # Arguments
        image: single channel array where each value represents the class key.
        label_values
    # Returns
        Colour coded image for segmentation visualization
    """
    colour_codes = np.array(label_values)
    x = colour_codes[image.astype(int)]

    return x
    
def plot_predictions(X_test,preds,path):
    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12,5))
    ax[0].imshow(X_test,aspect="auto")
    ax[0].set_title("Input")
    ax[1].imshow(preds, aspect="auto")
    ax[1].set_title("Prediction")
    fig.tight_layout()

#Make predictions
y_prob = model.predict(training_set) #Non corretto training, solo per visualizzare

#Output the most probable classes based on predictions
y_classes = y_prob.argmax(axis=-1)

#Reshape our classes in order to proceed with visualizing our predictions
y_classes = y_classes.reshape(y_classes.shape[0],360,480)

segment = colour_code_segmentation(y_classes,label_values)
#segment=np.squeeze(segment,axis=3)

plot_predictions(training_set[1],y_classes[1],path="3") #Non corretto training, solo per visualizzare